In [1]:
import torch
import torch_geometric
import os 
from dataset import GraphDataset
from model.hand_made import GNNPolicy
from readdataUC import UC
import pandas as pd
from scipy.sparse import coo_matrix
from torch.optim.lr_scheduler import CosineAnnealingLR
import time

In [2]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#set folder
model_save_path = './model_save/3bin_50_pretrain_routine_new/'
log_save_path = "./model_save/3bin_50_log_routine_new/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)
log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-5
NB_EPOCHS =9999
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-6

In [4]:
train_data = GraphDataset('dataset_20_0_1_w/3bin/train/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_data = GraphDataset('dataset_20_0_1_w/3bin/valid/')
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict)
#     print("载入模型成功")


In [6]:

def get_edge_index(N,T):

    source_node=[]
    target_node=[]
    for i in range(N):
        for t in range(T):
            j=i*T+t
            source_node.append(j)
            target_node.append(j)
            source_node.append(j+N*T)
            target_node.append(j)
            source_node.append(j+2*N*T)
            target_node.append(j)
            source_node.append(j+3*N*T)
            target_node.append(j)
            source_node.append(j+4*N*T)
            target_node.append(j)
            for ii in range(N):
                if ii != i:
                    source_node.append(j)
                    target_node.append(ii*T+t)
            for tt in range(T):
                if tt != t:
                    source_node.append(j)
                    target_node.append(i*T+tt)
    edge_index=torch.tensor([source_node,target_node], dtype=torch.long).to(DEVICE)
    return edge_index


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            # print(batch.variable_features.shape)
            N=int(batch.variable_features.shape[0]/24/5)
            T=24
            edge_index=get_edge_index(N,T)
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
                edge_index
            )
            
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            # print(batch.variable_features.shape)

            loss = loss_fn(presol, weighted_sol)

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [8]:
optimizer = torch.optim.Adam(PredictModel.parameters(), lr=LEARNING_RATE)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS, eta_min=ETA_MIN)
best_val_loss = 0.2

In [9]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.7f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.7f}")
    if valid_loss<best_val_loss:
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.6140075	Epoch 0 Valid loss: 0.4053737
Epoch 1 Train loss: 0.3882457	Epoch 1 Valid loss: 0.3442393
Epoch 2 Train loss: 0.3494121	Epoch 2 Valid loss: 0.3355641
Epoch 3 Train loss: 0.3412656	Epoch 3 Valid loss: 0.3297509
Epoch 4 Train loss: 0.3352983	Epoch 4 Valid loss: 0.3257476
Epoch 5 Train loss: 0.3320484	Epoch 5 Valid loss: 0.3225281
Epoch 6 Train loss: 0.3292454	Epoch 6 Valid loss: 0.3209221
Epoch 7 Train loss: 0.3271575	Epoch 7 Valid loss: 0.3220335
Epoch 8 Train loss: 0.3252082	Epoch 8 Valid loss: 0.3167484
Epoch 9 Train loss: 0.3238845	Epoch 9 Valid loss: 0.3173222
Epoch 10 Train loss: 0.3228662	Epoch 10 Valid loss: 0.3229238
Epoch 11 Train loss: 0.3209126	Epoch 11 Valid loss: 0.3163633
Epoch 12 Train loss: 0.3212210	Epoch 12 Valid loss: 0.3296327
Epoch 13 Train loss: 0.3191000	Epoch 13 Valid loss: 0.3117047
Epoch 14 Train loss: 0.3169312	Epoch 14 Valid loss: 0.3072703
Epoch 15 Train loss: 0.3134950	Epoch 15 Valid loss: 0.3047516
Epoch 16 Train loss: 0.31130

KeyboardInterrupt: 